In [2]:
import camelot
import pandas as pd
import openpyxl
import os
import pdfplumber
import re
from datetime import datetime
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [8]:
# columns = {                                                                                                   
#   "impactGWP100_kgCO2e": "(^GWP - Total$)|(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
#   "impactODP_kgCFC11e": "(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
#   "impactAP_kgSO2e": "(^AP)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
#   "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
#   "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
#   "impactADPElements_kgSbe": "(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
#   "impactADPFossilFuels_MJ": "(^ADPF)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
#   "renewablesUsedAsEnergy_MJ": "(^PERE)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
#   "renewablesUsedAsMaterial_MJ": "(^PERM)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
#   "embodiedRenewableEnergy_MJ": "(^PERT)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
#   "nonRenewablesUsedAsEnergy_MJ": "(^Non\xadrenewable primary energy as energy carrier$)|(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
#   "nonRenewablesUsedAsMaterial_MJ": "(^Non\xadrenewable primary energy as material utilization$)|(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
#   "embodiedEnergy_MJ": "(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
#   "recyclingMaterialUse_kg": "(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
#   "renewableRecylingFuelUse_MJ": "(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
#   "nonRenewableRecylingFuelUse_MJ": "(^Use of non\xadrenewable secondary fuels$)|(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
#   "cleanWaterNetUse_m3": "(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
#   "wasteHazardous_kg": "(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
#   "wasteNonHazardous_kg": "(^Non\xadhazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
#   "wasteRadioactive_kg": "(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
#   "reusableMaterialsOutput_kg": "(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
#   "recyclableMaterialsOutput_kg": "(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
#   "energyMaterialsOutput_kg": "(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
#   "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
#   "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
#   "exportedEnergyOutput_MJ": "(^EE$)",
    
#   "traciGWP_kgCO2e": 8,
#   "traciODP_kgCFC11e": 12,
#   "traciAP_kgSO2e": 6,
#   "traciEP_kgNe": 7,
#   "traciPOCP_kgO3e": 10,
#   "traciNRPE_MJ": 17
# }

columns = {
    "impactGWP100_kgCO2e_total": '(^GWP - Total)|(^GWP- \ntotal$)',
    "impactGWP100_kgCO2e_fossil": '(^GWP - Fossil)|(^GWP-\nfossil$)',
    "impactGWP100_kgCO2e_biogenic": '(^GWP - Biogenic)|(^GWP-\nbiogenic$)',
    "impactGWP100_kgCO2e_luluc": '(^GWP - Luluc)|(^GWP- \nluluc$)',
    "impactODP_WMO2014_kgCFC11e": '^ODP',
    "impactAP_molHeq": '^AP',
    "impactEPFreshwater_kgPO4e": '*EP - Freshwater',
    "impactEPFreshwater_kgP": '^EP - Freshwater',
    "impactEPMarine_kgNe": '(^EP - Marine)|(^EP- \nmarine$)',
    "impactEPTerrestrial_molNe": '(^EP - Terrestrial)|(^EP-\nterrestrial$)',
    "impactPOCP_kgNMVOCe": '^POCP',
    "cleanWaterDeprivation_m3": '^WDP',
    "particulateMatter_incidence": '^PM',
    "ionisingRadiation_kgU235eq": '^IR',
    "ecoToxicityFreshwater_CTUe": '^ETP - FW',
    "humanToxicityCancer_CTUh": '^HTTP - C',
    "humanToxicityNonCancer_CTUh": '^HTTP - NC',
    "potentialSoilQualityIndex": '^SQP',
  "impactGWP100_kgCO2e": "(^GWP - Total$)|(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^ADP-\nminerals)|(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^ADP-\nfossil)|(^ADPF)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^Non\xadrenewable primary energy as energy carrier$)|(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^Non\xadrenewable primary energy as material utilization$)|(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^Use of non\xadrenewable secondary fuels$)|(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^Non\xadhazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
  "recyclableMaterialsOutput_kg": "(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE$)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

headers = [
    "impactGWP100_kgCO2e_total",
    "impactGWP100_kgCO2e_fossil",
    "impactGWP100_kgCO2e_biogenic",    
    'impactGWP100_kgCO2e_luluc',
    'impactODP_WMO2014_kgCFC11e',
    'impactAP_molHeq',
    'impactEPFreshwater_kgPO4e',
    'impactEPFreshwater_kgP',
    'impactEPMarine_kgNe',
    'impactEPTerrestrial_molNe',
    'impactPOCP_kgNMVOCe',
    'cleanWaterDeprivation_m3',
    'particulateMatter_incidence',
    'ionisingRadiation_kgU235eq',
    'ecoToxicityFreshwater_CTUe',
    'humanToxicityCancer_CTUh',
    'humanToxicityNonCancer_CTUh',
    'potentialSoilQualityIndex',
    
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

In [9]:
def epd_number(file_name):
    with pdfplumber.open('./Group 3/' + file_name) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        item = re.findall('EPD registration number.*', text)
        try:
            r = item[0].split(' ')[3]
        except:
            r = '' 
        
    return r

In [10]:

def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'Envirodec_' + datetime.now().strftime("%Y-%m-%d") + '.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = epd_number(file_data["file_name"])
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-12-20/Envirodec/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            try:
                                value = float(replaced.replace('\xad', '-'))
                            except:
                                value = float(replaced.replace('\n', ''))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [11]:
def clean_df(df):
    transformed = df.applymap(lambda x: x.split(' ') if ('E-' in x) or ('E+' in x) or (x == '') else x)
    for i in range(transformed.shape[0]):
        row = transformed.iloc[[i]]
        for index, element in row.iloc[0].items():
            if element == ['']:
                for index_s, search in row.iloc[0].items():
                    if (len(search) > 1) and ('[kg' not in search) and (type(search) == list):
                        if index < index_s:        
                            transformed.loc[i,index] = [search[0]]
                            row.iloc[0][index] = [search[0]]
                            del search[0]
                        else: 
                            transformed.loc[i,index] = [search[1]]                            
                            row.iloc[0][index] = [search[1]]
                            del search[1]
                        break
    return transformed.applymap(lambda x: x[0] if type(x) == list else x)



In [12]:
files = os.listdir('./Group 3')
# files = ['15230.pdf', '14983.pdf.pdf', '14956.pdf']
r = []
for file in files:
    print("************ FILE NAME: " + file + " ****************")
    tables = camelot.read_pdf('./Group 3/' + file, pages='all',process_background=True)
    dataFrames = []

    file_info = {}
    stages = []
    for table in tables:
        df = table.df
        stages_added = False
        #stages = ['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
        #stages = ['A1-A3', 'A4', 'A1-A4']
        #stages = ['A1-A2', 'A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']
        gwp = ['GWP - Total', 'GWP- \ntotal']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:

                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                print(1)
                dataFrames.append(df)


        pere = ['PERE' ]
        for p in pere:
            if p in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(2)

        hwd = ['HWD']
        for h in hwd:
            if h in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(3)

                
    result = pd.concat(dataFrames)

    file_info["dataFrame"] = result

        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    

make_excel_file(r)


#r

************ FILE NAME: epd5054.pdf ****************


1
2
************ FILE NAME: epd5058.pdf ****************


1
2
************ FILE NAME: epd5059.pdf ****************


1
2
************ FILE NAME: epd5060.pdf ****************


1
2
************ FILE NAME: epd5061.pdf ****************


...

**************************************ERRROR*****************************
epd5058.pdf
D
^EP - Freshwater
**************************************ERRROR*****************************
epd5058.pdf
D
^PM
**************************************ERRROR*****************************
epd5058.pdf
D
^IR
**************************************ERRROR*****************************
epd5058.pdf
D
^ETP - FW
**************************************ERRROR*****************************
epd5058.pdf
D
^HTTP - C
**************************************ERRROR*****************************
epd5058.pdf
D
^HTTP - NC
**************************************ERRROR*****************************
epd5058.pdf
D
^SQP
**************************************ERRROR*****************************
epd5058.pdf
D
(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)
**************************************ERRROR*****************************
epd5058.pdf
D
(^Eutrophication, fraction of

**************************************ERRROR*****************************
epd5059.pdf
D
(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd5059.pdf
D
(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd5059.pdf
D
(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**************************************ERRROR*****************************
epd5059.pdf
D
(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)
**************************************ERRROR*****************************
epd5059.pdf
D
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd5059.pdf
D
(^EE$)
**************************************ERRROR*****************************
ep

**************************************ERRROR*****************************
epd5061.pdf
A1
*EP - Freshwater
**************************************ERRROR*****************************
epd5061.pdf
A1
^EP - Freshwater
**************************************ERRROR*****************************
epd5061.pdf
A1
^PM
**************************************ERRROR*****************************
epd5061.pdf
A1
^IR
**************************************ERRROR*****************************
epd5061.pdf
A1
^ETP - FW
**************************************ERRROR*****************************
epd5061.pdf
A1
^HTTP - C
**************************************ERRROR*****************************
epd5061.pdf
A1
^HTTP - NC
**************************************ERRROR*****************************
epd5061.pdf
A1
^SQP
**************************************ERRROR*****************************
epd5061.pdf
A1
(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)
***

**************************************ERRROR*****************************
epd5062.pdf
A1
*EP - Freshwater
**************************************ERRROR*****************************
epd5062.pdf
A1
^EP - Freshwater
**************************************ERRROR*****************************
epd5062.pdf
A1
^PM
**************************************ERRROR*****************************
epd5062.pdf
A1
^IR
**************************************ERRROR*****************************
epd5062.pdf
A1
^ETP - FW
**************************************ERRROR*****************************
epd5062.pdf
A1
^HTTP - C
**************************************ERRROR*****************************
epd5062.pdf
A1
^HTTP - NC
**************************************ERRROR*****************************
epd5062.pdf
A1
^SQP
**************************************ERRROR*****************************
epd5062.pdf
A1
(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)
***

**************************************ERRROR*****************************
epd5062.pdf
D
(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd5062.pdf
D
(^EE$)
**************************************ERRROR*****************************
epd5062.pdf
D
8
**************************************ERRROR*****************************
epd5062.pdf
D
12
**************************************ERRROR*****************************
epd5062.pdf
D
6
**************************************ERRROR*****************************
epd5062.pdf
D
7
**************************************ERRROR*****************************
epd5062.pdf
D
10
**************************************ERRROR*****************************
epd5062.pdf
D
17


In [11]:

tables = camelot.read_pdf('./Group 3/epd4112.pdf', pages='all', process_background=True)
print(len(tables))

7


In [17]:
tables[4].df

,0,1,2,3,4,5,6,7,8
0,Impact category,Unit\nA1-A3\nA4\nC1\nC2\nC3\nC4\nD,,,,,,,
1,GWP - Fossil,19.2\n0.2\n28.8 x10-6\n16.3 x10-3\n0\n2.03 x10...,,,,,,,
2,GWP - Biogenic,-65.9 x10-3\n43.1 x10-6\n0.87 x10-6\n4.85 x10-...,,,,,,,
3,GWP - Luluc,kg CO2 eq,25 x10-3,58.7 x10-6,62.1 x10-9,4.84 x10-9,0,1.65 x10-6,-5.73 x10-3
4,GWP - Total,kg CO2 eq,19.1,0.2,29.7 x10-6,16.3 x10-3,0,2.05 x10-3,-4.44
5,ODP,kg CFC11 eq,0.76 x10-6,45.2 x10-9,2.22 x10-12,3.74 x10-9,0,0.25 x10-9,-0.16 x10-6
6,AP,mol H+ eq,0.1,0.86 x10-3,0.22 x10-6,46.9 x10-6,0,13.8 x10-6,-23.9 x10-3
7,EP - Freshwater,kg P eq,7.73 x10-3,15.8 x10-6,29.8 x10-9,1.33 x10-6,0,0.7 x10-6,-1.82 x10-3
8,*EP - Freshwater,kg PO4 eq,23.6 x10-6,48.3 x10-6,91.1 x10-9,4.06 x10-6,0,2.14 x10-6,-5.58 x10-3
9,EP - Marine,kg N eq,17.4 x10-3,0.25 x10-3,29.9 x10-9,8.98 x10-6,0,3.24 x10-6,-3.91 x10-3
